# Python + Pandas is love

L'idéal pour une rapport de données, la validation d'une intuition, vite fait, sur un coin de table, en mode oneshot / quick&dirty ? ~~Excel~~ [Pandas](https://pandas.pydata.org/) !

Nous allons suivre la génération d'un rapport à partir d'un _petit_ dump de données, ce qui va nous permettre de faire une visite guidée des fonctionnalités de pandas. Ce ne sera pas exhaustif, mais devrait vous permettre d'y penser la prochaine fois. 

(Version page web)[https://nbviewer.jupyter.org/github/flo-dhalluin/python-pandas-talk/blob/master/pandas-talk-run.ipynb]

## La base


In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np

### Serie

Une série, est une séquence de valeurs, de type homogène (toutes du même type : String, entiers, flotants, dates ... ), __ avec un index __

In [ ]:
s = pd.Series([5,4,3,2,1,2,3,4,5])
s_datetimes = pd.date_range(start="2017-01-01", periods=86400, freq="2H").to_series()

#display(s.head())
s_datetimes.head()

## Dataframe

Un Dataframe, c'est un tableau : 
- un ensemble de Series, nommées ( les colonnes)
- qui partagent un index


In [ ]:
data = pd.DataFrame({"time": s_datetimes[:120].values,  # get rid of the index
                     "value": np.random.random(120),
                     "category": pd.Categorical(list("abcdef" * 20))})

print(data.info(()))
display(data.head())

### Import/Export

L'idée étant évidemment d'importer nos données ici, et pas de les générer. Pandas supporte à peu près tout les fichiers formats de données courants :
- csv 
- excel 
- hdf5 
- json/msgpack/parquet

_mais aussi_  on peut créer un dataframe directement depuis une requête SQL, ou même Big Query.

Nous allons ici utiliser la lingua franca de la data ( rappel du contexte : on est sale, on est rapide, on est pas sexys ...) : le csv. 

In [ ]:
clients = pd.read_csv("client_files.csv", delimiter=";", index_col=0, parse_dates=[1])
#display(clients.head(4))
events = pd.read_csv("events.csv", delimiter=";", index_col=0, parse_dates=True)
#display(events.info())
print("events is row x cols", events.shape)
print("columns are : ", events.columns)
print("time range:", events.index.min(), events.index.max())
display(events.head(4))

## Let's go

### Les données :

Les clients remplissent des dossiers pour soucrire à des produits, ils doivent envoyer certains justificatifs qui sont
  validés automatiquement (ou pas). events enregistre les différentes soumissions de documents. (client_uuid, correspond à un dossier). comme la validation automatique peut échouer, le client resoumet ces pièces.

### Le problème
Nous voudrions compiler un peu des stats sur le nombre de soumission, par type de dossier ... juste à partir de la table des evenements.


### Requêtes simples

In [ ]:
import datetime

cni_events = events[events.doc_type == "CNI"] # keep only CNI type docs
# keep only events occuring on january 10th. 
day_events = events[events.index.date == datetime.date(2017,1,10)]

print("%d events recorded on January 10th" % len(day_events))

# By Index : .ix  - by "position" .iloc 
events.iloc[14:16]

In [ ]:
# Pour des requêtes encore plus péchues 

events.query('doc_type=="RIB" & status & index > "20170110"').head()


### GroupBy

> Tableau croisé dynamique, c'est mieux si on en reste là...

Passons dans le vif du sujet, et moulinons un peu nos data.

In [ ]:
# comptons les soumissions de documents par type/status. 
(events.groupby(('doc_type','status')) # groupage imbriqué 
       .count())   # fonction d'aggregation 


### Split - apply/transform - combine

Insérons ici _une jolie image_ pour illuster le flux de données

In [ ]:
# utiliseons groupby pour "numéroter" les soumissions par utilisateurs

# cleanup
events['doc_type'] = events['doc_type'].fillna('NO_DOC')
events["timestamp"] = events.index # --- groupby transform pas très heureux avec les index

# groupons par dossier, puis chaque document
grouped_doc = events.groupby(('client_uuid','doc_type'))

# on transforme une colonne. ( Split - apply - combine)
events['doc_submission_count'] = grouped_doc["timestamp"].transform(np.argsort)
events['time_since_first'] = events.groupby('client_uuid')["timestamp"].transform(lambda ts: ts-ts[0])

events.head()

In [ ]:
# Nb moyen de soumissions de doc avant succés : PAR Type de doc.
(events[events.status==True] # seulement les soumissions ok
     .groupby('doc_type')    # toujours le petit groupby ... 
     .doc_submission_count   # quantième soumission
     .aggregate(['mean', 'max', 'min'])) # aggregation des stats.. 

### Des graphiques ! 

In [ ]:
# temps passé par dossier

(events.groupby('client_uuid') # Tu vois ce que je veux dire ? 
     .time_since_first # juste le temps depuis la première action
     .max().map(lambda x: x.total_seconds()) # timedelta -> "float" ( seconds )
     .hist(bins=50)) # KABOOM !


### Encore plus de trucs fun: join et pivot. 

... To be continued

- [sources](https://github.com/flo-dhalluin/python-pandas-talk)
- fait avec [jupyter / notebook](https://jupyter.org/)
- Le saviez vous ? vous pouvez exporter un notebook en slides/reveal.js
